In [1]:
            # Application
# Sentiment analysis
# customer Support 
# Language detection
# fakenews detection

            # Pipleine
# text acquire
# preprocessing
# Vectolization
# Modeling ( common --> naive bayes , random forest)
# Evalutations
# Deploy


In [2]:
import nltk 
import numpy as np
import pandas as pd

In [15]:
df_temp = pd.read_csv('Emotion_Dataset/text_emotion.csv')
df_temp.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [25]:
df = df_temp.iloc[:10000].copy()
df.head()
df.info()

df.isnull().sum()

df['sentiment'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   10000 non-null  int64 
 1   sentiment  10000 non-null  object
 2   author     10000 non-null  object
 3   content    10000 non-null  object
dtypes: int64(1), object(3)
memory usage: 312.6+ KB


sentiment
worry         3115
sadness       2216
neutral       1857
surprise       562
hate           535
happiness      469
love           369
relief         227
fun            211
empty          194
enthusiasm     132
boredom         69
anger           44
Name: count, dtype: int64

In [31]:
# Preprocessing

# remove puncation

import string
exclude = string.punctuation

def remove_pun(text):
    return text.translate(str.maketrans('' , '' , exclude))

df['content']  = df['content'].apply(remove_pun)
df['content'].iloc[:4]


0    tiffanylue i know  i was listenin to bad habit...
1    Layin n bed with a headache  ughhhhwaitin on y...
2                        Funeral ceremonygloomy friday
3                  wants to hang out with friends SOON
Name: content, dtype: object

In [33]:
# remove stop words 

from nltk.corpus import stopwords

def remove_stopwords(text):
    dummy = []
    for word in text.split():
        if word in stopwords.words('english'):
            dummy.append("")
        else:
            dummy.append(word)
    x = dummy[:]
    dummy.clear()
    return " " .join(x)

df['content']  = df['content'].apply(remove_stopwords)
df['content'].iloc[:4]

0    tiffanylue  know   listenin  bad habit earlier...
1           Layin n bed   headache ughhhhwaitin   call
2                        Funeral ceremonygloomy friday
3                           wants  hang   friends SOON
Name: content, dtype: object

In [35]:
# convert to lower case

df['content']  = df['content'].str.lower()
df['content'].iloc[:4]

0    tiffanylue  know   listenin  bad habit earlier...
1           layin n bed   headache ughhhhwaitin   call
2                        funeral ceremonygloomy friday
3                           wants  hang   friends soon
Name: content, dtype: object

In [55]:
# Modeling

X = df['content']
y = df['sentiment']

In [56]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y = lb.fit_transform(y)

y.shape

(10000,)

In [71]:
# train test split
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X , y , test_size= 0.2 , random_state=1)

y_train[0:5]
X_train[:5]

X_train.shape

(8000,)

In [79]:
# Vectorization(bag or words)

from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features= 5000)


X_train_bow = bow.fit_transform(X_train).toarray()
x_test_bow = bow.transform(X_test).toarray()

X_train_bow.shape , x_test_bow.shape

((8000, 5000), (2000, 5000))

In [80]:
# Naive bayes fucntion

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score , confusion_matrix

gb = GaussianNB()

gb.fit(X_train_bow , y_train)

GaussianNB()

In [82]:
y_pred = gb.predict(x_test_bow)

print(accuracy_score(y_test , y_pred))

confusion_matrix(y_test , y_pred)

0.147


array([[  0,   0,   0,   1,   0,   0,   2,   1,   2,   0,   0,   0,   1],
       [  2,   1,   1,   0,   3,   0,   1,   0,   1,   0,   1,   0,   3],
       [  2,   0,   1,   1,   3,   5,   4,   5,   6,   1,   4,   0,   7],
       [  0,   0,   0,   2,   0,   2,   0,   1,   6,   0,   7,   2,   3],
       [  0,   0,   0,   3,   2,   2,   4,   4,  14,   1,   7,   4,   5],
       [  1,   1,   3,   2,   6,  14,   7,   7,  16,   7,  14,   3,   9],
       [  0,   3,   3,   2,   1,  11,  10,   3,  29,   1,  17,   2,  18],
       [  1,   2,   0,   6,   9,   8,   2,   5,  12,   4,  13,   5,  10],
       [ 26,  18,  23,  35,  27,  23,  19,  21,  71,  20,  54,  22,  32],
       [  3,   1,   0,   3,   2,   3,   2,   3,  11,   2,   9,   3,   8],
       [  6,  12,  15,  22,  25,  28,  30,  31,  69,  20,  86,  22,  58],
       [  4,   1,   4,   6,   5,   8,  11,   2,  17,   4,  13,   5,  11],
       [ 15,  17,  20,  29,  40,  44,  47,  38, 128,  18, 117,  41,  95]],
      dtype=int64)

In [83]:
from sklearn.ensemble import RandomForestClassifier

rcf = RandomForestClassifier()
rcf.fit(X_train_bow , y_train)



RandomForestClassifier()

In [84]:
y_pred = rcf.predict(x_test_bow)

print(accuracy_score(y_test , y_pred))

confusion_matrix(y_test , y_pred)

0.3025


array([[  0,   0,   0,   0,   0,   0,   1,   1,   1,   0,   1,   0,   3],
       [  0,   0,   1,   0,   0,   0,   0,   0,   2,   0,   2,   1,   7],
       [  0,   0,   3,   0,   0,   0,   1,   0,   7,   0,   9,   2,  17],
       [  1,   0,   0,   0,   0,   1,   0,   1,   7,   0,   2,   0,  11],
       [  0,   0,   0,   0,   0,   0,   0,   0,  15,   1,  10,   0,  20],
       [  0,   0,   1,   0,   0,   5,   0,   2,  17,   4,  18,   3,  40],
       [  0,   0,   1,   0,   0,   2,  10,   0,  24,   0,  27,   2,  34],
       [  1,   0,   0,   0,   0,   1,   1,   4,  17,   1,  13,   2,  37],
       [  0,   0,   7,   0,   0,   5,   5,   2, 168,   5,  67,  10, 122],
       [  0,   0,   0,   1,   0,   1,   0,   0,  10,   1,  12,   1,  24],
       [  1,   0,   4,   0,   1,   3,   5,   5,  91,   2, 113,  13, 186],
       [  2,   0,   2,   0,   0,   2,   2,   1,  34,   0,   8,   8,  32],
       [  3,   0,   2,   4,   1,   9,  10,   4, 167,   6, 141,   9, 293]],
      dtype=int64)

In [90]:
# use tf - idf for vectorlization

from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(max_features= 5000)

X_train_tf= tf_idf.fit_transform(X_train).toarray()
x_test_tf = tf_idf.transform(X_test).toarray()

In [91]:
rcf = RandomForestClassifier()
rcf.fit(X_train_tf , y_train)
y_pred = rcf.predict(x_test_tf)

print(accuracy_score(y_test , y_pred))

confusion_matrix(y_test , y_pred)

0.3255


array([[  0,   0,   0,   0,   0,   0,   1,   0,   1,   0,   2,   0,   3],
       [  0,   0,   0,   0,   0,   0,   1,   0,   4,   0,   2,   0,   6],
       [  0,   0,   0,   0,   0,   0,   1,   0,  10,   0,   8,   1,  19],
       [  0,   0,   0,   0,   0,   0,   0,   1,   4,   0,   2,   0,  16],
       [  0,   0,   0,   0,   0,   0,   0,   0,  10,   0,   8,   0,  28],
       [  0,   0,   1,   0,   0,   1,   0,   4,  13,   0,  22,   1,  48],
       [  0,   0,   0,   0,   0,   0,  11,   0,  26,   0,  24,   0,  39],
       [  0,   0,   0,   0,   1,   0,   2,  11,  14,   0,  11,   1,  37],
       [  0,   0,   3,   0,   0,   1,   7,   5, 134,   3,  56,  10, 172],
       [  0,   0,   0,   1,   0,   1,   1,   2,  11,   1,  12,   0,  21],
       [  0,   2,   0,   0,   0,   3,   9,   7,  61,   0, 123,   6, 213],
       [  1,   0,   0,   0,   0,   1,   3,   1,  23,   0,  10,   6,  46],
       [  0,   0,   0,   2,   0,   3,   9,   2, 142,   1, 122,   4, 364]],
      dtype=int64)

In [ ]:
# ensemale techniques
# heuristic featues
# ml --> deep learning
# imbalace datset (xx)